## ATAC + MULTIOME (females september 2021)

### Matching genes to proximal peaks

A common approach to start peak - gene correlation analysis is to find all peaks within 50kb of a gene. Here we build an adjacency matrix matching peak to genes.



In [1]:
import numpy as np
import scanpy as sc 
import pandas as pd
import anndata
import anndata2ri ## For sparse matrix conversion from r 2 py

**r2py setup**

In [2]:
import rpy2.rinterface_lib.callbacks
import logging

In [3]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

Loading the rpy2 extension enables cell magic to be used. This runs R code in jupyter notebook cells.

In [4]:
%load_ext rpy2.ipython

**Load data**

In [5]:
# Define variables
outdir = "/nfs/team292/vl6/my_MULTIOME_dir/females_july2021/"
experiment_prefix = 'females_'

In [6]:
adata = sc.read_h5ad(outdir + experiment_prefix + "_ATAC.wCisTopic.h5ad")

In [7]:
adata

AnnData object with n_obs × n_vars = 90920 × 153356
    obs: 'cellatac_clusters', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'multiome_GermCells', 'multiome_Somatic', 'code', 'sample', 'sex', 'stage', 'individual'
    var: 'peak_width', 'exon', 'gene', 'promoter', 'annotation', 'gene_name', 'gene_id', 'tss_distance', 'ENCODE_blacklist', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'cellatac_clusters_colors', 'individual_colors', 'multiome_GermCells_colors', 'multiome_Somatic_colors', 'neighbors', 'sample_colors', 'stage_colors', 'umap'
    obsm: 'X_cistopic_46', 'X_umap'
    layers: 'binary_raw'
    obsp: 'connectivities', 'distances'

In [8]:
peaks = adata.var_names

**Match peaks to genes**

In [9]:
%%R 
library(Matrix)
library(GenomicRanges)
library(ensembldb)
library(EnsDb.Hsapiens.v86) ## Remember to pick your genome!
library(tidyr)
# library(Signac)

In [10]:
%%R
## String - GRanges conversion
## Borrowed from Signac functions 
## https://satijalab.org/signac/reference/GRangesToString.html
StringToGRanges <- function(regions, sep = c("-", "-"), ...) {
  ranges.df <- data.frame(ranges = regions)
  ranges.df <- separate(
    data = ranges.df,
    col = "ranges",
    sep = paste0(sep[[1]], "|", sep[[2]]),
    into = c("chr", "start", "end")
  )
  granges <- makeGRangesFromDataFrame(df = ranges.df, ...)
  return(granges)
}

GRangesToString <- function(grange, sep = c("-", "-")) {
  regions <- paste0(
    as.character(x = seqnames(x = grange)),
    sep[[1]],
    start(x = grange),
    sep[[2]],
    end(x = grange)
  )
  return(regions)
}

# Extend genomicRanges
# 
extend <- function(x, upstream=0, downstream=0)     
{
    if (any(strand(x) == "*"))
        warning("'*' ranges were treated as '+'")
    on_plus <- strand(x) == "+" | strand(x) == "*"
    new_start <- start(x) - ifelse(on_plus, upstream, downstream)
    new_end <- end(x) + ifelse(on_plus, downstream, upstream)
    ranges(x) <- IRanges(new_start, new_end)
    trim(x)
}


# Find peaks close to features of interest
#
# @param peaks_gr GenomicRanges object containing peaks
# @param features_gr GenomicRanges object containing features (e.g. genes)
# @param d distance to include peak, in bps (default 50000)
# @param feat_anno column in `features_gr@elementMetadata` containing annotation to name features (if NULL converts Granges to string)
#
# @return Sparse adjacency matrix indicating hits
peak2feature <- function(peaks_gr, features_gr, d=50000, feat_anno=NULL){
  seqlevelsStyle(features_gr) <- seqlevelsStyle(peaks_gr)
  
  ## Find peaks overlapping the search range around the features
  ext_gr <- extend(features_gr, upstream = d, downstream = d)
  ovs <- findOverlaps(peaks_gr, ext_gr)
  
  ## Define identifiers for peaks and features
  all_peaks <- GRangesToString(peaks_gr, sep = c(":", '-'))
  if (is.null(feat_anno)) {
    all_feats <- GRangesToString(features_gr, sep = c(":", '-'))
  } else {
    all_feats <- features_gr@elementMetadata[[feat_anno]]
  }
  
  ## Build adjacency matrix for hits
  adj_mat <- Matrix::Matrix(data=0, nrow = length(all_peaks), ncol=length(all_feats))
  for (i in unique(subjectHits(ovs))) {
    # if (length(adj_mat[queryHits(ovs[subjectHits(ovs)==i]),i]) > 0) {
    adj_mat[queryHits(ovs[subjectHits(ovs)==i]),i] <- 1
    # }
  }
  colnames(adj_mat) <- all_feats
  rownames(adj_mat) <- all_peaks
  
  adj_mat
  
}

In [11]:
%%R  -i peaks -o adj_mat
genes_gr <- genes(EnsDb.Hsapiens.v86)
peaks_gr <- StringToGRanges(peaks, sep=c(":", "-"))

## Compute peak2gene adjacency matrix
adj_mat <- peak2feature(peaks_gr, genes_gr, feat_anno = "gene_id", d=50000)

In [12]:
%%R -o genes
genes <- colnames(adj_mat)

In [13]:
adj_mat

<rpy2.robjects.methods.RS4 object at 0x7f3e6df4e340> [RTYPES.S4SXP]
R classes: ('dgCMatrix',)

In [14]:
## Convert sparse matrix w anndata2ri
adj_mat = anndata2ri.r2py.rmat_to_spmat(adj_mat)

In [15]:
adata.varm["peak2gene"] = adj_mat
adata.uns["peak2gene_genes"] = genes

In [16]:
del adata.uns["peak2gene_genes"]

**Save anndata with cisTopic**

In [17]:
adata.write_h5ad(outdir + experiment_prefix + "_ATAC.wCisTopic.h5ad")

In [6]:
adata = sc.read(outdir + experiment_prefix + "_ATAC.wCisTopic.h5ad")

In [18]:
adata.var['peaks_formatted'] = [i.replace(":", "-") for i in adata.var_names]

In [19]:
adata.var = adata.var.set_index('peaks_formatted')
adata.var.to_csv(outdir + experiment_prefix + "adata_var_for_cicero.csv")

In [20]:
adata.varm["peak2gene"].shape

(153356, 63970)

In [21]:
genes

'ENSG0000...,'ENSG0000...,'ENSG0000...,...,'ENSG0000...,'ENSG0000...,'ENSG0000...


In [22]:
np.unique(adata.obs['individual'])

array(['F81', 'Hrv11', 'Hrv12', 'Hrv13', 'Hrv39', 'Hrv49', 'Hrv50',
       'Hrv58', 'Hrv59', 'Hrv65', 'Hrv91', 'Hrv92'], dtype=object)

In [23]:
adata.var['chromosome'] = [i.split("-")[0] for i in adata.var_names]
adata.var['chromosome'].value_counts(dropna = False)

chr1     14075
chr2     12371
chr3      9898
chr6      8804
chr5      8318
chr7      7935
chr11     7920
chr10     7800
chr4      7585
chr12     7284
chr8      6886
chr9      6813
chr17     6730
chr16     5365
chr15     5128
chr19     4858
chr14     4763
chr20     4538
chr13     4251
chr18     3538
chr22     3319
chrX      3242
chr21     1934
chrY         1
Name: chromosome, dtype: int64

In [24]:
adata.var[adata.var['chromosome'] == "chrY"].index

Index(['chrY-6533757-6534449'], dtype='object', name='peaks_formatted')

#### Only 1 read from chromosome Y